# TruthfulQA

TruthfulQA differs quite a bit from the other tasks as there is not a single correct answer. Interestingly, all the true responses come before the false responses.

In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import datasets
import os

API_KEY = os.environ["ZENO_API_KEY"]

Feel free to change the list of models used.
You can go to the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) to check what models are available.
Some of them might not have associated data, you can check this by clicking on the little icon next to the model name.
If you get a 404 after clicking, we won't be able to fetch the model data and this notebook will crash.

In [ ]:
models = ["meta-llama/Llama-2-70b-hf", "mistralai/Mistral-7B-v0.1", "tiiuae/falcon-40b", "Riiid/sheep-duck-llama-2-70b-v1.1", "AIDC-ai-business/Marcoroni-70B-v1", "ICBU-NPU/FashionGPT-70B-V1.1", "adonlee/LLaMA_2_70B_LoRA", "uni-tianyan/Uni-TianYan"]

In [ ]:
def get_data(model: str):
    data_path = "details_" + model.replace("/", "__")
    return datasets.load_dataset(
        "open-llm-leaderboard/" + data_path,
        "harness_truthfulqa_mc_0",
    )

In [ ]:
def generate_dataset(df):
    df_lim = df[["question", "mc2_targets"]]
    df_lim.loc[:, "data"] = df_lim.apply(lambda x: "\n" + x["question"] + "\n\n" + "\n".join(map(lambda y: "- " + str(y).lstrip(), x["mc2_targets"]["choices"])), axis=1)
    df_lim.loc[:, "# of options"] = df_lim.apply(lambda x: len(x["mc2_targets"]["choices"]), axis=1)
    df_lim.loc[:, "# of answers"] = df_lim.apply(lambda x: x["mc2_targets"]["labels"].sum(), axis=1)
    df_lim.loc[:, "label"] = df_lim["mc2_targets"].apply(lambda r: "\n" + "\n".join(["True" if s == 1 else "False" for s in r["labels"]]))
    df_lim = df_lim.drop(columns=["mc2_targets", "question"])
    df_lim["id"] = df_lim.index 
    return df_lim

def generate_system(df):
    df_system = df[["predictions", "mc2", "mc1_targets"]]
    df_system["predictions"] = df_system.apply(lambda x: f"mc2: {x['mc2']}" + "\n\n" + "\n".join(map(lambda y: str(round(y, 2)), x['predictions'][len(x['mc1_targets']['choices']):])), axis=1)
    df_system["id"] = df_system.index
    return df_system[["predictions", "mc2", "id"]]

Make sure you have your Zeno API key in your environment variables.

In [ ]:
client = ZenoClient(API_KEY)

Lets create a project to hold the data for the TruthfulQA task.

In [ ]:
proj = client.create_project(
    name="TruthfulQA", 
    view="text-classification", 
    description="TruthfulQA (https://arxiv.org/abs/2109.07958) task in the Open-LLM-Leaderboard.",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["mc2"])
    ]
)

Let us now upload the data to the project we just created.

In [ ]:
df = generate_dataset(get_data(models[0])['latest'].to_pandas())
print("\nYour dataset has {} rows\n".format(len(df)))
num_rows = len(df)
proj.upload_dataset(df, id_column="id", label_column="label", data_column="data")

Finally, let us upload all the model outputs for the models we specified above.

In [ ]:
for model in models:
    dataset = get_data(model)['latest'].to_pandas()
    print(dataset.columns)
    if len(dataset) != num_rows:
        print("Skipping {} because it has {} rows instead of {}".format(model, len(dataset), num_rows))
        continue
    df_system = generate_system(dataset)
    proj.upload_system(df_system[["predictions", "mc2", "id"]], name=model.replace('/', "__"), output_column="predictions", id_column="id")